# Import Required Libraries
Import libraries such as transformers for GPT-2 and BART, and requests for calling the Ollama API.

In [56]:
# Import Required Libraries
from transformers import GPT2LMHeadModel, GPT2Tokenizer, BartForConditionalGeneration, BartTokenizer
import requests
import json
from prettytable import PrettyTable


# Load Models
Load GPT-2 and BART models locally, and set up the API call for the Ollama model.

In [41]:
# Load Models
# Load GPT-2
gpt2_model = GPT2LMHeadModel.from_pretrained("/home/xgen0/CertyChain/CertyLex/models/torch/boe_gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("/home/xgen0/CertyChain/CertyLex/models/torch/boe_gpt2")

# Load BART
bart_model = BartForConditionalGeneration.from_pretrained("/home/xgen0/CertyChain/CertyLex/models/torch/boe_bart")
bart_tokenizer = BartTokenizer.from_pretrained("/home/xgen0/CertyChain/CertyLex/models/torch/boe_bart")

# Ollama API setup
ollama_url = "http://127.0.0.1:11434/api/generate"  # Replace with actual Ollama API endpoint

# Prepare Input Data
Prepare the input text to be used for all three models.

In [31]:
# Prepare Input Data
input_text = """
    "Describe en detalle las principales características de la Constitución Española, incluyendo su estructura, principios fundamentales y derechos reconocidos."
"""

# Tokenize input for GPT-2
gpt2_inputs = gpt2_tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)

# Tokenize input for BART
bart_inputs = bart_tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)

# Generate Model Outputs
Generate outputs from GPT-2, BART, and multiple Ollama models.

In [51]:
# Generate Model Outputs
# GPT-2 output
gpt2_outputs = gpt2_model.generate(
    gpt2_inputs["input_ids"],
    attention_mask=gpt2_inputs["attention_mask"],  # Pass attention mask
    max_length=1024,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=gpt2_tokenizer.eos_token_id  # Set pad_token_id to eos_token_id
)
gpt2_result = gpt2_tokenizer.decode(gpt2_outputs[0], skip_special_tokens=True)

# BART output
bart_outputs = bart_model.generate(
    bart_inputs["input_ids"],
    attention_mask=bart_inputs["attention_mask"],  # Pass attention mask
    max_length=1024,
    num_beams=4,
    early_stopping=True,
    pad_token_id=bart_tokenizer.eos_token_id  # Set pad_token_id to eos_token_id
)
bart_result = bart_tokenizer.decode(bart_outputs[0], skip_special_tokens=True)

# Ollama outputs
ollama_models = [#"deepseek-r1:32b",
                 "deepseek-r1:8b", "llama3.2:3b"]
ollama_results = {}

for model in ollama_models:
    try:
        response = requests.post(ollama_url, json={"model": model, "prompt": input_text}, stream=True)
        response.raise_for_status()  # Raise an error for bad HTTP responses
        
        # Collect all fragments of the response
        full_response = ""
        for line in response.iter_lines():
            if line:
                fragment = json.loads(line)
                full_response += fragment.get("response", "")
        
        ollama_results[model] = full_response
    except requests.exceptions.RequestException as e:
        ollama_results[model] = f"Error: Unable to connect to {model} - {str(e)}"

# Compare Outputs
Compare the outputs of GPT-2, BART, and the multiple Ollama models.

In [52]:
# Compare Outputs
print("GPT-2 Output:\n", gpt2_result)
print("\nBART Output:\n", bart_result)

for model, result in ollama_results.items():
    print(f"\n{model} Output:\n{'-'*20}\n{result}\n")

GPT-2 Output:
 Describe en detalle las principales características de la Constitución Española, incluyendo su estructura, principios fundamentales y derechos reconocidos.
En el artículo 3.1 de la Constitución Española, se establece que las principales características de la Constitución Española se rige por el Real Decreto 463/2020, de 14 de marzo, por el que se declara el estado de alarma para la gestión de la situación de crisis sanitaria ocasionada por el COVID-19, se hace necesario adaptar la disposición adicional primera de la Ley 47/2003, de 26 de noviembre, General Presupuestaria, a través de la Ley 40/2015, de 1 de octubre, de Régimen Jurídico del Sector Público, a la Ley 50/1997, de 27 de noviembre, del Gobierno, a la Ley 55/2000, de 1 de diciembre, del Estatuto Marco del personal estatutario de los servicios públicos de salud, aprobado por el Real Decreto Legislativo 5/2015, de 30 de octubre.
En el artículo 3.2 de la Constitución Española, se establece que las principales cara

# Visualize Results
Visualize the differences in the outputs using text-based or graphical methods.

In [53]:
# Visualize Results
# Example: Print outputs side by side for comparison
outputs = {
    "GPT-2": gpt2_result,
    "BART": bart_result,
    **ollama_results  # Unpack Ollama results into the dictionary
}

for model, output in outputs.items():
    print(f"\n{model} Output:\n{'-'*20}\n{output}\n")


GPT-2 Output:
--------------------
Describe en detalle las principales características de la Constitución Española, incluyendo su estructura, principios fundamentales y derechos reconocidos.
En el artículo 3.1 de la Constitución Española, se establece que las principales características de la Constitución Española se rige por el Real Decreto 463/2020, de 14 de marzo, por el que se declara el estado de alarma para la gestión de la situación de crisis sanitaria ocasionada por el COVID-19, se hace necesario adaptar la disposición adicional primera de la Ley 47/2003, de 26 de noviembre, General Presupuestaria, a través de la Ley 40/2015, de 1 de octubre, de Régimen Jurídico del Sector Público, a la Ley 50/1997, de 27 de noviembre, del Gobierno, a la Ley 55/2000, de 1 de diciembre, del Estatuto Marco del personal estatutario de los servicios públicos de salud, aprobado por el Real Decreto Legislativo 5/2015, de 30 de octubre.
En el artículo 3.2 de la Constitución Española, se establece que

# Define Example Prompts
Provide a set of 10 well-structured and context-rich prompts in Spanish to generate outputs from the models for comparison.

In [54]:
# Define Example Prompts
example_prompts = [
    "Describe en detalle las principales características de la Constitución Española, incluyendo su estructura, principios fundamentales y derechos reconocidos.",
    "Resume las últimas actualizaciones publicadas en el BOE relacionadas con las leyes laborales, especificando los cambios más relevantes y su impacto en los trabajadores y empleadores.",
    "Genera un modelo de carta formal para solicitar una subvención gubernamental, incluyendo los apartados necesarios como introducción, justificación y conclusión.",
    "Explica el proceso completo para registrar una nueva empresa en España, detallando los pasos legales, administrativos y fiscales que deben seguirse.",
    "Proporciona un análisis detallado de las reformas educativas más recientes en España, destacando los objetivos, cambios implementados y su impacto en el sistema educativo.",
    "Redacta un aviso legal para un conflicto de propiedad, incluyendo los argumentos legales y las referencias normativas aplicables en España.",
    "Resume los puntos clave de las últimas regulaciones medioambientales publicadas en el BOE, explicando su propósito y las obligaciones que imponen a las empresas.",
    "Genera un comunicado de prensa para anunciar una nueva política gubernamental, destacando sus objetivos, beneficios esperados y el público objetivo.",
    "Explica los derechos y responsabilidades de los ciudadanos españoles según la Constitución, incluyendo ejemplos prácticos de cómo se aplican en la vida cotidiana.",
    "Redacta una propuesta detallada para un nuevo proyecto de infraestructura pública, incluyendo objetivos, presupuesto estimado y beneficios para la comunidad."
]

# Print prompts to verify
for i, prompt in enumerate(example_prompts, 1):
    print(f"Prompt {i}: {prompt}")

Prompt 1: Describe en detalle las principales características de la Constitución Española, incluyendo su estructura, principios fundamentales y derechos reconocidos.
Prompt 2: Resume las últimas actualizaciones publicadas en el BOE relacionadas con las leyes laborales, especificando los cambios más relevantes y su impacto en los trabajadores y empleadores.
Prompt 3: Genera un modelo de carta formal para solicitar una subvención gubernamental, incluyendo los apartados necesarios como introducción, justificación y conclusión.
Prompt 4: Explica el proceso completo para registrar una nueva empresa en España, detallando los pasos legales, administrativos y fiscales que deben seguirse.
Prompt 5: Proporciona un análisis detallado de las reformas educativas más recientes en España, destacando los objetivos, cambios implementados y su impacto en el sistema educativo.
Prompt 6: Redacta un aviso legal para un conflicto de propiedad, incluyendo los argumentos legales y las referencias normativas a

# Generate Outputs for All Prompts
Iterate over the example prompts and generate outputs for each model (GPT-2, BART, and Ollama models).

In [58]:
# Generate Outputs for All Prompts Sequentially
all_outputs = {
    "GPT-2": [],
    "BART": [],
    "Ollama": {model: [] for model in ollama_models}
}

for i, prompt in enumerate(example_prompts, 1):
    print(f"Processing Prompt {i}: {prompt}\n")

    # GPT-2 output
    gpt2_inputs = gpt2_tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    gpt2_outputs = gpt2_model.generate(
        gpt2_inputs["input_ids"],
        attention_mask=gpt2_inputs["attention_mask"],
        max_length=1024,
        num_return_sequences=1,
        temperature=0.7,
        pad_token_id=gpt2_tokenizer.eos_token_id
    )
    gpt2_result = gpt2_tokenizer.decode(gpt2_outputs[0], skip_special_tokens=True)
    all_outputs["GPT-2"].append(gpt2_result)
    print(f"GPT-2 Output for Prompt {i}: {gpt2_result}\n")

    # BART output
    bart_inputs = bart_tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    bart_outputs = bart_model.generate(
        bart_inputs["input_ids"],
        attention_mask=bart_inputs["attention_mask"],
        max_length=1024,
        num_beams=4,
        early_stopping=True,
        pad_token_id=bart_tokenizer.eos_token_id
    )
    bart_result = bart_tokenizer.decode(bart_outputs[0], skip_special_tokens=True)
    all_outputs["BART"].append(bart_result)
    print(f"BART Output for Prompt {i}: {bart_result}\n")

    # Ollama outputs
    for model in ollama_models:
        try:
            response = requests.post(ollama_url, json={"model": model, "prompt": prompt}, stream=True)
            response.raise_for_status()

            # Collect all fragments of the response
            full_response = ""
            for line in response.iter_lines():
                if line:
                    fragment = json.loads(line)
                    full_response += fragment.get("response", "")

            if model not in ollama_results:
                ollama_results[model] = []  # Initialize as a list if not already
            if model not in ollama_results or not isinstance(ollama_results[model], list):
                ollama_results[model] = []  # Initialize as a list if not already
            ollama_results[model].append(full_response)
            print(f"{model} Output for Prompt {i}: {full_response}\n")
        except requests.exceptions.RequestException as e:
            error_message = f"Error: Unable to connect to {model} - {str(e)}"
            all_outputs["Ollama"][model].append(error_message)
            print(f"{model} Error for Prompt {i}: {error_message}\n")
    # Store results in all_outputs
    all_outputs["Ollama"][model].append(full_response)
    

Processing Prompt 1: Describe en detalle las principales características de la Constitución Española, incluyendo su estructura, principios fundamentales y derechos reconocidos.

GPT-2 Output for Prompt 1: Describe en detalle las principales características de la Constitución Española, incluyendo su estructura, principios fundamentales y derechos reconocidos.
En el artículo 3.1 de la Constitución Española, se establece que las principales características de la Constitución Española se rige por el Real Decreto 463/2020, de 14 de marzo, por el que se declara el estado de alarma para la gestión de la situación de crisis sanitaria ocasionada por el COVID-19, se hace necesario adaptar la disposición adicional primera de la Ley 47/2003, de 26 de noviembre, General Presupuestaria, a través de la Ley 40/2015, de 1 de octubre, de Régimen Jurídico del Sector Público, a la Ley 50/1997, de 27 de noviembre, del Gobierno, a la Ley 55/2000, de 1 de diciembre, del Estatuto Marco del personal estatutari

# Display Outputs in a Structured Format
Format the outputs of all models for better comparison across the 10 prompts.

In [59]:

# Create a table to display outputs
output_table = PrettyTable()
output_table.field_names = ["Prompt #", "Model", "Output"]

# Add GPT-2 outputs
for i, output in enumerate(all_outputs["GPT-2"], 1):
    output_table.add_row([i, "GPT-2", output])

# Add BART outputs
for i, output in enumerate(all_outputs["BART"], 1):
    output_table.add_row([i, "BART", output])

# Add Ollama outputs
for model, outputs in all_outputs["Ollama"].items():
    for i, output in enumerate(outputs, 1):
        output_table.add_row([i, model, output])

# Print the table
print(output_table)

+----------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------